In [23]:
import sys
import requests
import multiprocessing as mp
import redis

from flask import Flask, request

def create_app(db_num):
    app = Flask(__name__)
    r = redis.Redis(db=db_num)
    r.mset({"db": "0"}) 
    

    @app.route("/get", methods=['get'])
    def get():
        db = r.get("db") 
        return db
    
    
    @app.route("/set", methods=['post'])
    def set():
        if len(request.form) == 0:
            return "400: Bad Request"
        item = request.form["item"]
        db = r.get("db")
        new_db = db.decode('utf-8')
        new_db += ',' + item
        r.mset({"db":new_db})
        return "200: OK" 
    
    @app.route("/hello", methods=['get'])
    def hello():
        return "Hello"
    return app


    #app.run(debug=False, host='0.0.0.0', port=port,use_reloader=False,threaded=True)
    
def make_leader(app,port,db_num): 
    r = redis.Redis(db=db_num)
    @app.route("/replicate", methods=['post','get'])
    def replicate():
        ''' send contents to another node '''

        if len(request.form) == 0:
            return "400: BAD Request"
        
        port = request.form["port"]
        url="http://127.0.0.1:"+port+"/store"
        db = r.get("db")
        print(db.decode('utf-8'))
        x = requests.post(url, data={'item': db.decode('utf-8')})

        return x.text

    app.run(debug=False, host='0.0.0.0', port=port,use_reloader=False,threaded=True)


def make_follower(app,port,db_num):
    r = redis.Redis(db=db_num)
    @app.route("/store", methods=['post'])
    def store():
        ''' receive db contents from another node '''
        if len(request.form) == 0:
            return "400: Bad Request"
        
        item = request.form["item"]
        print("item:"+item)
        r.mset({"db":item})

        return '200: OK'
    app.run(debug=False, host='0.0.0.0', port=port,use_reloader=False,threaded=True)
    


In [24]:
servers = []
num_servers = 2
for i in range(num_servers):
    app = create_app(i)
    p = None
    if i == num_servers - 1:
        p = mp.Process(target=make_leader, args=(app,3000+i,i,))
    else:
        p = mp.Process(target=make_follower, args=(app,3000+i,i,))
    p.start()
    servers.append(p)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:3001/ (Press CTRL+C to quit)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Jun/2020 23:38:20] "POST /set HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 23:38:21] "POST /set HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 23:38:23] "POST /set HTTP/1.1" 200 -


0,2,3,4
item:0,2,3,4


127.0.0.1 - - [07/Jun/2020 23:38:24] "POST /store HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 23:38:24] "POST /replicate HTTP/1.1" 200 -
127.0.0.1 - - [07/Jun/2020 23:38:35] "GET /get HTTP/1.1" 200 -


In [22]:
for s in servers:
    s.terminate()